In [1]:
import os, sys
os.environ['ON_PY']="1"
from data_utils import CaseGenerator
import random
from gym_jobshop.envs.fjsp_env_2 import FJSPEnv
from stable_baselines3 import PPO


In [2]:

def test(model_name, test_job_length, test_op_pt, ):
    # 创建测试环境
    test_env = FJSPEnv(num_jobs=3, num_machines=2)
    obs = test_env.set_initial_data(test_job_length, test_op_pt)
    model = PPO.load("../train/" + model_name)
    # print("Initial Observation:", obs)

    # 测试模型
    done = False
    total_reward = 0
    step_count = 0

    while not done:
        # 使用模型预测动作
        action, _states = model.predict(obs, deterministic=True)
        # 环境执行动作
        obs, reward, terminated, truncated, info = test_env.step(action)
        done = terminated  or truncated # 判断是否结束
        total_reward += reward
        step_count += 1

        # 可视化每步的结果（可选）
        print(f"Step: {step_count}, Action: {action}, Reward: {reward}, Info: {info}")

    print(f"Test completed in {step_count} steps with total reward: {total_reward}")
    test_env.render()


In [3]:
n_j=14
n_m=11
op_per_job_min = 10
op_per_job_max = 20
case = CaseGenerator(n_j, n_m, op_per_job_min, op_per_job_max,
                        flag_same_opes=False)
JobLength, OpPT, _ = case.get_case(0)
print(JobLength)
print(OpPT)
test("ppo_fjsp",JobLength, OpPT)

199
[15 11 16 10 11 18 20 18 13 12 13 16 11 15]
[[ 5  5  0 ...  6  5  6]
 [ 0  0 15 ...  0  0 16]
 [ 0 14 19 ... 19 14 19]
 ...
 [ 0  0  0 ...  0  0 11]
 [ 0 17  0 ...  0  0 16]
 [ 0  0  0 ...  0  0 18]]
Step: 1, Action: 61, Reward: 0.0, Info: {'makespan': 175, 'next_schedule_time': 0.0, 'step_count': 1, 'chosem_op': 63, 'chosem_mch': 6}
Step: 2, Action: 10, Reward: 0.0, Info: {'makespan': 175, 'next_schedule_time': 0.0, 'step_count': 2, 'chosem_op': 0, 'chosem_mch': 10}
Step: 3, Action: 40, Reward: -10.0, Info: {'invalid_action': "FJSP_Env.py Error from choosing action: Op 42 can't be processed by Mch 7"}
Test completed in 3 steps with total reward: -10.0
None


TypeError: object of type 'NoneType' has no len()